In [3]:
import pandas as pd
import json
# Read in the data

# to load the sample data we need pandas
df = pd.read_csv('test_data.csv')
sample_route_data = df.to_dict('records')
# sample_route_data_json = df.to_json(orient='index',index=['direction_id','stop_sequence'],indent=4)

sample_route_data_json = df.groupby('direction_id').apply(lambda x: x.to_json(orient='records'))


d = {'departure_times':'departure_time'}

# works 2/6/2023
# j = df.groupby(['day_type']).apply(lambda x: x[['departure_times']].rename(columns=d).to_dict('records')).reset_index(name='depature_times').to_json('test_json3.json',orient='records',indent=4)
data_by_direction_id = df.groupby('direction_id').apply(lambda x: x.to_json(orient='records'))
group_direction_id_by_stop_id = df.groupby(['direction_id','stop_id']).apply(lambda x: x.to_json(orient='records'))

group_direction_id_by_stop_id_by_day_type = df.groupby(['direction_id','stop_id','day_type']).apply(lambda x: x.rename(columns=d).to_dict('records'))
group_direction_id_by_day_type_by_stop_sequence = df.groupby(['direction_id','day_type','stop_sequence']).apply(lambda x: x.rename(columns=d).to_dict('records'))

D = (group_direction_id_by_stop_id_by_day_type
  .groupby(level=0)
  .apply(lambda df: df.xs(df.name).to_dict())
  .to_dict()
)


# PANDAS IS VERY HELPFUL WHEN YOU ARE DOING:
#  - A JOIN (merging data from different tables)
#  - A GROUP BY (aggregating data by a certain column)
#  - TO ADD DATA TO A DATABASE QUICKLY


# 1. group data by direction_id
# 2. for each direction_id, group data by stop_id
# 3. for each stop_id, group data by day type <-- store this data as an object called "departure times"


# d = {'departure_times':'departure_time'}
# df.rename(columns=d, inplace=True)
# works as of 2.6.2023
d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       )
# group_direction_id_by_stop_id_by_day_type
group_direction_id_by_day_type_by_stop_sequence

C:\Users\hikou\AppData\Local\Temp\ipykernel_28508\113217929.py:44: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']


KeyError: "Columns not found: 'departure_time'"

In [143]:
group_direction_id_by_stop_id_by_day_type.head(5)

def clean_up_group(x):
    #  x.set_index('day_type')['data'].to_dict()
     all_data = x.to_dict('records')
#      print(all_data)
     return all_data


# the function works!! stopped here 2/9/2023
# we need to get this to run on the entire dataframe in the larger script
def get_departure_times(target_data):
       initial_json = (target_data.groupby(['direction_id','day_type'])['departure_times','stop_sequence','stop_name','stop_id','coordinates']
       .apply(lambda x: x.to_dict('records'))
       .reset_index()
       .rename(columns={0:'data'})
       .groupby('direction_id')['day_type','data']
       .apply(lambda x:x.set_index('day_type')['data'].to_dict())
       .to_json(orient='index',indent=4)
       )
       cleaned_json = [json.loads(initial_json)]
       # initial_json.to_dict()
       return cleaned_json

final_result = pd.DataFrame()
final_result['route_code'] = df['route_code'].unique().astype('str')
final_result['route_id'] = df['route_id'].unique().astype('str')
final_result['payload'] = get_departure_times(df)

print(final_result.to_json('final_result.json',orient='records',indent=4))

test_json_10 = (df.groupby(['direction_id','day_type'])['departure_times','stop_sequence','stop_name','stop_id','coordinates']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby('direction_id')['day_type','data']

       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .to_json('test_json10.json',orient='index',indent=4)
       )
get_departure_times(df)
# d = (df.groupby(['direction_id','day_type','stop_sequence'])['departure_time','day_type']
#        .apply(lambda x: x.to_dict('r'))
#        .reset_index(name='data')
#        .groupby(['direction_id','day_type'])['stop_sequence','data']
#        .apply(lambda x: x.set_index('stop_sequence')['data'].to_dict())
#        .reset_index(name='departure_times')
#        .to_json('test_json10.json',orient='records',indent=4)
#        )


None


C:\Users\hikou\AppData\Local\Temp\ipykernel_28508\4132807081.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  initial_json = (target_data.groupby(['direction_id','day_type'])['departure_times','stop_sequence','stop_name','stop_id','coordinates']
C:\Users\hikou\AppData\Local\Temp\ipykernel_28508\4132807081.py:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  test_json_10 = (df.groupby(['direction_id','day_type'])['departure_times','stop_sequence','stop_name','stop_id','coordinates']
C:\Users\hikou\AppData\Local\Temp\ipykernel_28508\4132807081.py:33: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  .apply(lambda x: x.to_dict('r'))


[{'0': {'saturday': [{'departure_times': "['06:07:00', '06:14:00', '06:20:00', '06:25:00', '06:30:00', '06:35:00', '06:36:00', '06:36:00', '06:41:00', '06:47:00', '06:52:00', '06:53:00', '06:53:00', '06:57:00', '07:02:00', '07:07:00', '07:10:00', '07:10:00', '07:12:00', '07:17:00', '07:22:00', '07:27:00', '07:27:00', '07:27:00', '07:32:00', '07:37:00', '07:42:00', '07:42:00', '07:43:00', '07:48:00', '07:54:00', '07:55:00', '07:55:00', '08:00:00', '08:05:00', '08:07:00', '08:07:00', '08:10:00', '08:15:00', '08:18:00', '08:18:00', '08:20:00', '08:25:00', '08:29:00', '08:29:00', '08:30:00', '08:35:00', '08:40:00', '08:40:00', '08:40:00', '08:45:00', '08:50:00', '08:51:00', '08:51:00', '08:55:00', '09:00:00', '09:03:00', '09:03:00', '09:05:00', '09:10:00', '09:15:00', '09:16:00', '09:16:00', '09:20:00', '09:25:00', '09:26:00', '09:26:00', '09:30:00', '09:34:00', '09:34:00', '09:36:00', '09:41:00', '09:41:00', '09:44:00', '09:49:00', '09:49:00', '09:52:00', '09:57:00', '09:57:00', '09:59:00

In [15]:
# print(group_direction_id_by_stop_id_by_day_type.to_frame())

# resource from stackoverflow: https://stackoverflow.com/questions/52923685/convert-pandas-multiindex-series-to-json-python
# method a
final_result = (df.groupby(['direction_id','stop_id'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby('direction_id')['stop_id','data']
       .apply(lambda x: x.set_index('stop_id')['data'].to_dict())
       .to_json('test_json8.json',orient='index',indent=4)
       )

# method b

d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
       .to_json('test_json6.json',orient='index',indent=4)
       )


print(d)
# final_result

None


C:\Users\hikou\AppData\Local\Temp\ipykernel_11060\3185241020.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_result = (df.groupby(['direction_id','stop_id'])['departure_time','day_type']
C:\Users\hikou\AppData\Local\Temp\ipykernel_11060\3185241020.py:6: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  .apply(lambda x: x.to_dict('r'))
C:\Users\hikou\AppData\Local\Temp\ipykernel_11060\3185241020.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
C:\Users\hikou\AppData\Local\Temp\ipykernel_11060\3185241020.py:16: FutureWarning: Using short name for 'orient' is deprecated. O

In [18]:
# new code
d = {'departure_times':'departure_time'}

def process_data(x):
    # x.drop('day_type', axis=1, inplace=True)  
    # print(x.day_type)
    result = x.set_index('day_type')['data'].to_dict()
    # result.to_json('test_reulst.json',orient='index',indent=4)
    return result

df.rename(columns=d, inplace=True)
d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: process_data(x))
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
    #    .to_json('test_json8.json',orient='index',indent=4)
       )

e = d.groupby(['direction_id'])['stop_id','departure_times'].apply(lambda x: x.set_index('stop_id')['departure_times'].to_dict()).reset_index(name='stop_id')
e.to_json('test_json6.json',orient='records',indent=4)


C:\Users\hikou\AppData\Local\Temp\ipykernel_11060\3936196280.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
C:\Users\hikou\AppData\Local\Temp\ipykernel_11060\3936196280.py:13: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  .apply(lambda x: x.to_dict('r'))
C:\Users\hikou\AppData\Local\Temp\ipykernel_11060\3936196280.py:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  e = d.groupby(['direction_id'])['stop_id','departure_times'].apply(lambda x: x.set_index('stop_id')['departure_times'].to_dict()).reset_index(name='stop_id')


In [1]:
d = {'departure_times':'departure_time'}
df.rename(columns=d, inplace=True)
# dataframe_by_direction_id = df.groupby('direction_id').apply(lambda x: x.to_dict('r')).to_frame()
# dataframe_by_direction_id = dataframe_by_direction_id.rename(columns={0:'route_id'}).apply(lambda x: x.to_dict('r')).to_frame()
# print(dataframe_by_direction_id)
# from collections import defaultdict

d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
       # .to_frame(orient='index')
       # .to_json('test_json6.json',orient='index',indent=4)
       )

# print(d)
d


NameError: name 'df' is not defined

In [5]:
# group_direction_id_by_stop_id_by_day_type = df.groupby(['direction_id','stop_id','day_type']).apply(lambda x: x.rename(columns=d).to_dict('records'))


# need to get the direction_id to be the key at the top level
# need to get the stop_id to be the key at the second level - done
# need to get the day_type to be the key at the third level - done
# need to get the departure_time to be the value at the fourth level - done



d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
       # .to_json('test_json6.json',orient='index',indent=4)
       )


C:\Users\hikou\AppData\Local\Temp\ipykernel_26204\4227211953.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
C:\Users\hikou\AppData\Local\Temp\ipykernel_26204\4227211953.py:10: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  .apply(lambda x: x.to_dict('r'))


TypeError: unhashable type: 'dict'

### Goal `json` v2

``` js
{
    route_id: '',
    '0': {
        weekday: [
            {
                stop_sequence: 1,
                stop_id: '',
                stop_name: '',
                coordinate: '',
                departure_times: ['', '', '']
            },{
                stop_sequence: 2,
                stop_id: '',
                stop_name: '',
                coordinate: '',
                departure_times: ['', '', '']
            },{
                stop_sequence: 3,
                stop_id: '',
                stop_name: '',
                coordinate: '',
                departure_times: ['', '', '']
            }
        ],
        saturday: [

        ],
        sunday: [

        ]
    },
    '1': {
        
    }
```

### The goal `json`

``` js
{
  route_id: '',
  direction_0:
  [
    {
      stop_id: '',
      stop_name: '',
      stop_sequence: '',
      coordinate: '',
      departure_times: 
      {
        weekday:
        [
          {
            departure_time: ''
          },{
            departure_time: ''
          },{
            departure_time: ''
          },{
            departure_time: ''
          }
        ],
        saturday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ],
        sunday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ]
      }
    },{
      stop_id: '',
      stop_name: '',
      stop_sequence: '',
      coordinate: '',
      departure_times: 
      {
        weekday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ],
        saturday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ],
        sunday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ]
      }
    }
  ],
  direction_1: 
  [
    
  ]
}

```